# 1. import Dependencies

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
import datetime
import numpy as np
import gradio as gr
import keras

## 2. Data-set loading and Pre-processing
Data-set link: https://mega.nz/file/zcdywLhI#fck4ufXy_o_Uiu0vGqh-cZiKHw5Xe_n4M2qWUWSheAI

In [2]:
# Set the path to the dataset folders
train_data_dir = 'archive (6)/train'
test_data_dir = 'archive (6)/val'

In [3]:
batch_size = 32
epochs = 15
# Set the input image dimensions
img_width, img_height = 150, 150

In [4]:
# Create an ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255,        # Normalize pixel values between 0 and 1
    shear_range=0.2,        # Apply random shear transformations
    zoom_range=0.2,         # Apply random zoom transformations
    horizontal_flip=True)  # Flip images horizontally


# Only normalize pixel values for testing
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


# Load and normalize testing data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


## 3. Model Building and Training

In [5]:
# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                           input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 128)               22429824  
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 22430849 (85.57 MB)
Trainable params: 22430849 (85.57 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [7]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [8]:
# Train the model
model.fit(train_generator, epochs=epochs, callbacks=[tensorboard_callback])

Epoch 1/15
277/277 [==============================] - 84s 301ms/step - loss: 0.7126 - accuracy: 0.5894
Epoch 2/15
277/277 [==============================] - 75s 271ms/step - loss: 0.6283 - accuracy: 0.6317
Epoch 3/15
277/277 [==============================] - 74s 268ms/step - loss: 0.6143 - accuracy: 0.6445
Epoch 4/15
277/277 [==============================] - 72s 259ms/step - loss: 0.5943 - accuracy: 0.6628
Epoch 5/15
277/277 [==============================] - 79s 282ms/step - loss: 0.5770 - accuracy: 0.6839
Epoch 6/15
277/277 [==============================] - 69s 250ms/step - loss: 0.5555 - accuracy: 0.7079
Epoch 7/15
277/277 [==============================] - 69s 248ms/step - loss: 0.5419 - accuracy: 0.7207
Epoch 8/15
277/277 [==============================] - 67s 243ms/step - loss: 0.5236 - accuracy: 0.7418
Epoch 9/15
277/277 [==============================] - 69s 249ms/step - loss: 0.5053 - accuracy: 0.7530
Epoch 10/15
277/277 [==============================] - 67s 240ms/step - l

In [9]:
model.save('x_ray.keras')

## 4. Deployment using Gradio

In [10]:
model = keras.models.load_model('x_ray.keras')

# Define the function for image classification
def classify_image(img):
    # Set the input image dimensions
    img_width, img_height = 150, 150

    # Resize the image to match the model's input shape
    img = img.resize((img_width, img_height))

    # Convert the image to a numpy array
    img = np.array(img)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)

    # Get the prediction
    prediction = model.predict(img)

    return "NOT fractured" if prediction > 0.5 else "fractured"


In [11]:
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil", label="Upload an X-ray image"),
    outputs="text",
    title="Bone Fracture Classification",
)

IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------


In [12]:
# Start the Gradio interface
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 101ms/step
